In [1]:
from dotenv import load_dotenv
import os

try:
    print(__file__)  # 检查 __file__ 是否存在, os.path.dirname() 可以返回指定路径的父目录
    dotenv_path = os.path.join(os.path.dirname(__file__), '.env')
except NameError:
    print("Running in an interactive environment, using current directory instead.")
    # 使用 os.path.join() 和 .. 返回指定路径的父目录
    cur_work_dir = os.getcwd()
    print(cur_work_dir)
    dotenv_path = os.path.normpath(os.path.join(cur_work_dir, '.env'))
print(dotenv_path)
load_dotenv(dotenv_path, override=True)

Running in an interactive environment, using current directory instead.
/Users/ericyoung/ysx/code/github-study/langChain-rookie
/Users/ericyoung/ysx/code/github-study/langChain-rookie/.env


True

# End-to-end agent  端到端代理
- 以下代码片段表示一个使用 LLM 来决定使用哪些工具的完整功能代理。
    - 它配备了一个通用搜索工具。
    - 它具有对话记忆功能，这意味着它可以作为多轮聊天机器人使用。
- Tavily（一个搜索引擎）作为工具

In [2]:
# Import relevant functionality
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatOllama(
    model="qwen2.5:latest",
    temperature=0.5,
    # other params...
)
search = TavilySearchResults(max_results=3)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [3]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [SystemMessage(content="answer in chinese"), HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in sf
================================== Ai Message ==================================

很高兴认识你，Bob！你想了解关于旧金山的哪些信息呢？如果你有任何具体问题或需要帮助的地方，请告诉我。


In [4]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (0acdadde-106c-4c08-a8a8-89c56ac2c7ed)
 Call ID: 0acdadde-106c-4c08-a8a8-89c56ac2c7ed
  Args:
    query: San Francisco weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1740233998, 'localtime': '2025-02-22 06:19'}, 'current': {'last_updated_epoch': 1740233700, 'last_updated': '2025-02-22 06:15', 'temp_c': 7.8, 'temp_f': 46.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}

In [5]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================

根据最新的数据，旧金山当前的气温为7.8℃（约46℉），天气状况是部分多云。风速约为3.4英里/小时（约5.4公里/小时），湿度为89%。

你可以访问 [WeatherAPI](https://www.weatherapi.com/) 获取更多详细的实时天气信息。

希望这些信息对你有帮助！如果还有其他问题，请随时告诉我。


# 一步一步实现

In [6]:
# %pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

## Define tools  定义工具
- 例如,选择的工具是 Tavily——一个搜索引擎。LangChain 内置了一个工具，可以轻松使用 Tavily 搜索引擎。

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1740233998, 'localtime': '2025-02-22 06:19'}, 'current': {'last_updated_epoch': 1740233700, 'last_updated': '2025-02-22 06:15', 'temp_c': 7.8, 'temp_f': 46.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 3.4, 'wind_kph': 5.4, 'wind_degree': 58, 'wind_dir': 'ENE', 'pressure_mb': 1024.0, 'pressure_in': 30.24, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 89, 'cloud': 50, 'feelslike_c': 7.1, 'feelslike_f': 44.8, 'windchill_c': 8.0, 'windchill_f': 46.4, 'heatindex_c': 8.1, 'heatindex_f': 46.5, 'dewpoint_c': 7.7, 'dewpoint_f': 45.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 6.5, 'gust_kph': 10.4}}"}, {'url': 'https://weathershogun.com/weather/usa/c

## Using Language Models  使用语言模型


In [8]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="qwen2.5:latest",
    temperature=0.5,
    # other params...
)

In [9]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

- 使用 .bind_tools 来让语言模型了解这些工具。

In [10]:
model_with_tools = model.bind_tools(tools)

- 调用这个模型了。我们先用一个普通的消息调用它，看看它会如何响应。我们可以查看 content 字段以及 tool_calls 字段。

In [14]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"response: {response.model_dump_json()}")
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

response: {"content":"Hello! How can I assist you today?","additional_kwargs":{},"response_metadata":{"model":"qwen2.5:latest","created_at":"2025-02-22T14:27:45.283187Z","done":true,"done_reason":"stop","total_duration":1572342917,"load_duration":35077500,"prompt_eval_count":189,"prompt_eval_duration":1108000000,"eval_count":10,"eval_duration":422000000,"message":{"role":"assistant","content":"Hello! How can I assist you today?","images":null,"tool_calls":null}},"type":"ai","name":null,"id":"run-03753139-9aeb-473a-9422-4c9c6576b9c2-0","example":false,"tool_calls":[],"invalid_tool_calls":[],"usage_metadata":{"input_tokens":189,"output_tokens":10,"total_tokens":199}}
ContentString: Hello! How can I assist you today?
ToolCalls: []


- 传入一些期望调用工具的输入，看看会发生什么。
    - 没有文本内容，但有一个工具调用！想调用 Tavily Search 工具。

In [15]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': 'ce8aecd4-e1c4-4239-908f-444a58beca32', 'type': 'tool_call'}]


## Create the agent  创建代理
- 使用 LangGraph 来构建代理
- 请注意，我们传入的是 model ，而不是 model_with_tools 。因为 create_react_agent 会在后台调用 .bind_tools 。

In [16]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent  运行代理
- 未调用任何工具

In [18]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='fd593de3-23e3-4974-a489-e903b671de62'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-02-22T14:36:17.147514Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4485121083, 'load_duration': 570761708, 'prompt_eval_count': 189, 'prompt_eval_duration': 3334000000, 'eval_count': 10, 'eval_duration': 427000000, 'message': Message(role='assistant', content='Hello! How can I assist you today?', images=None, tool_calls=None)}, id='run-59457ee0-03ea-48b7-95f9-4cc41fc8e311-0', usage_metadata={'input_tokens': 189, 'output_tokens': 10, 'total_tokens': 199})]

- 调用该工具

In [19]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='8f4dd6ff-49f1-42f7-999b-eaacecd1a657'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-02-22T14:37:06.316735Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4813110542, 'load_duration': 33790542, 'prompt_eval_count': 194, 'prompt_eval_duration': 1199000000, 'eval_count': 75, 'eval_duration': 3574000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-0da9d5dc-2c6a-47ee-96fd-39f9c7753df4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in san francisco'}, 'id': 'b9f96691-8184-450d-aa1e-b7f92d5f7051', 'type': 'tool_call'}], usage_metadata={'input_tokens': 194, 'output_tokens': 75, 'total_tokens': 269}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'Cali

## Streaming Messages  流式消息
- 显示中间进度，可以在消息发生时流式传输返回消息。

In [20]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (1668da3f-e1d2-4eaf-9092-6b7ad35456de)
 Call ID: 1668da3f-e1d2-4eaf-9092-6b7ad35456de
  Args:
    query: current weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1740235032, 'localtime': '2025-02-22 06:37'}, 'current': {'last_updated_epoch': 1740234600, 'last_updated': '2025-02-22 06:30', 'temp_c': 7.8, 'temp_f': 46.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1

## Streaming tokens  流式tokens
- 除了回传消息外，回传标记也很有用。我们可以通过指定 stream_mode="messages" 来实现
- message.text() ，这需要 langchain-core>=0.3.37 

In [21]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The current weather in San Francisco is partly cloudy with a temperature of 7.8°C (46°F). The wind speed is at 3.4 mph (5.4 km/h) coming from the northeast direction (ENE). The humidity level is 89%, and there's no precipitation expected.

If you're planning for an extended stay, February in San Francisco typically sees temperatures ranging from about 57°F to 61°F during the day with overnight lows around 46°F. The weather can be quite variable, so it might be a good idea to pack layers and have some rain gear handy as well.|

## Adding in memory  添加记忆功能
- 添加记忆功能，就需要传递一个检查点。
- 在调用代理时，我们还需要传递一个 thread_id （以便它知道从哪个线程/对话继续）。

In [22]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [23]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [24]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-02-22T14:42:35.601067Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2633524458, 'load_duration': 34469333, 'prompt_eval_count': 191, 'prompt_eval_duration': 2070000000, 'eval_count': 11, 'eval_duration': 522000000, 'message': Message(role='assistant', content='Hello Bob! How can I assist you today?', images=None, tool_calls=None)}, id='run-0cf005ed-b467-4422-9075-487c847baa8f-0', usage_metadata={'input_tokens': 191, 'output_tokens': 11, 'total_tokens': 202})]}}
----


In [25]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob!', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-02-22T14:42:54.52852Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1118915333, 'load_duration': 69745542, 'prompt_eval_count': 216, 'prompt_eval_duration': 789000000, 'eval_count': 6, 'eval_duration': 247000000, 'message': Message(role='assistant', content='Your name is Bob!', images=None, tool_calls=None)}, id='run-661d45d5-d687-474d-87b4-6ac40883ed42-0', usage_metadata={'input_tokens': 216, 'output_tokens': 6, 'total_tokens': 222})]}}
----


- 开始一个新的对话，你只需要改变使用的 thread_id 即可

In [26]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Since I don\'t have any information about you, you can tell me your name directly, or we can try to find it through other means. However, if you want me to generate a response as if I know your name, you could say something like "My name is [Your Name]". What would you like to do?', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-02-22T14:43:39.353217Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5035296250, 'load_duration': 35367125, 'prompt_eval_count': 192, 'prompt_eval_duration': 1654000000, 'eval_count': 68, 'eval_duration': 3339000000, 'message': Message(role='assistant', content='Since I don\'t have any information about you, you can tell me your name directly, or we can try to find it through other means. However, if you want me to generate a response as if I know your name, you could say something like "My name is [Your Name]". What would you like to do?', images=None, tool_calls=Non

# 总结
- 定义工具 tool
- 定义LLM model
- 创建reat Agent
- 添加历史记忆功能 memory
- 流式输出